In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import tensorflow as tf

from src.DataPreprocessor.data_preprocessor import DataPreprocessor, Mode
from src.LearningKeras.net_architecture import cnn_150x150x5,cnn_150x150x5_3class
from src.LearningKeras.train import KerasTrainer

from tqdm import trange

np.random.seed(1)
tf.set_random_seed(2)

check data_visualisation for details

In [2]:
from src.DataPreprocessor.DataIOBackend.gdal_backend import GdalBackend
from src.DataPreprocessor.data_visualiser import DataVisualiser

dataiobackend = GdalBackend()
data_preprocessor = DataPreprocessor(data_dir="data/Region 1 - Lopukangri/",
                              backend=dataiobackend,
                              filename_prefix="tibet",
                              mode=Mode.TRAIN,
                              seed=1)

## NN

### Train

In [3]:
model_generator = lambda: cnn_150x150x5_3class()
ensemble_size = 5
batch_size = 5

trainer = KerasTrainer(model_generator=model_generator,
                       ensemble_size=ensemble_size,
                       data_preprocessor=data_preprocessor,
                       batch_size=batch_size)
train_generator = data_preprocessor.train_generator(batch_size=batch_size,
                                         class_probabilities=[0.33, 0.33, 0.33],
                                         patch_size=(150, 150),
                                         channels=[0, 1, 2, 3, 4])

In [4]:
trainer.train(steps_per_epoch=50, epochs=5, train_generator=train_generator)

TypeError: 'generator' object is not callable

In [ ]:
for i in range(ensemble_size):
    pathlib.Path('models').mkdir(parents=True, exist_ok=True)
    trainer.models[i].save_weights('models/model_{}.h5'.format(i))

In [13]:
t = next(train_generator)

StopIteration: 